__Import Libraries__

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from bqplot import Bars, Lines, LinearScale, DateScale, Axis, Figure, Layout
from ipywidgets import VBox, HBox
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

### Load Dataset from Zillow Home Value Index
* Retrieved from https://www.zillow.com/research/data/
* Data Type is Single-Family Homes Times Series ($) by county US wide

In [2]:
# Load the dataset
file_path = 'County_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month_v2.csv'
data = pd.read_csv(file_path)

# Displaying the first few rows of the dataset to understand its structure
data.head()

,County & State,1/31/2000,2/29/2000,3/31/2000,4/30/2000,5/31/2000,6/30/2000,7/31/2000,8/31/2000,9/30/2000,...,5/31/2023,6/30/2023,7/31/2023,8/31/2023,9/30/2023,10/31/2023,11/30/2023,12/31/2023,1/31/2024,2/29/2024
0,"Los Angeles County, CA",218109.6614,218351.9636,219253.9924,221034.2609,223301.9647,225478.4711,227678.6538,229942.9407,232027.3310,...,840773.8285,847854.1806,859394.2974,873560.9233,887641.7893,899129.9503,907716.4206,913122.6128,911973.7283,906790.4414
1,"Cook County, IL",147964.6629,147921.7372,148148.3663,148818.4676,149663.9906,150546.8177,151356.5574,152568.0603,153990.8260,...,296293.3034,298020.2147,299950.9762,302194.0149,303945.6241,305305.3397,306242.5782,306888.8415,307391.7031,308718.4049
2,"Harris County, TX",111394.1238,111358.8246,111187.7213,111118.3715,111092.1685,111286.1650,111488.0642,111789.0796,112181.9393,...,282318.3407,282960.6208,283744.4065,284559.2247,284896.4113,284939.7274,284693.0418,284411.8285,284527.6504,285006.6967
3,"Maricopa County, AZ",147041.8392,147345.9207,147742.5134,148554.3669,149447.9413,150211.1292,150977.1668,151693.8225,152498.5159,...,461229.5436,463373.7563,466262.6341,469321.6095,471862.4581,474104.4184,475939.2821,476935.5727,477297.3014,477869.9382
4,"San Diego County, CA",230036.4263,230951.6382,232047.8538,234522.8778,237169.8082,240508.5797,244335.7027,248525.5830,252462.3305,...,903768.2136,914304.4490,926825.9847,939717.6219,952202.7165,962435.9478,969407.6635,973071.9064,975520.7101,980746.2117


## Process and Transform Data

*Transform the data from wide to a long format using the melt function so that each row represent an individual observation in the time series data*

In [3]:
data_long = data.melt(id_vars=['County & State'], var_name='Date', value_name='MedianHousePrice')

In [4]:
data_long.head()

,County & State,Date,MedianHousePrice
0,"Los Angeles County, CA",1/31/2000,218109.6614
1,"Cook County, IL",1/31/2000,147964.6629
2,"Harris County, TX",1/31/2000,111394.1238
3,"Maricopa County, AZ",1/31/2000,147041.8392
4,"San Diego County, CA",1/31/2000,230036.4263


*Convert the **Date** column to a datetime type*

In [5]:
data_long['Date'] = pd.to_datetime(data_long['Date'], format='%m/%d/%Y')

*Split County & State feature into two feature columns*

In [6]:
data_long[['County', 'State']] = data_long['County & State'].str.split(', ', expand=True)

In [7]:
data_long.head()

,County & State,Date,MedianHousePrice,County,State
0,"Los Angeles County, CA",2000-01-31,218109.6614,Los Angeles County,CA
1,"Cook County, IL",2000-01-31,147964.6629,Cook County,IL
2,"Harris County, TX",2000-01-31,111394.1238,Harris County,TX
3,"Maricopa County, AZ",2000-01-31,147041.8392,Maricopa County,AZ
4,"San Diego County, CA",2000-01-31,230036.4263,San Diego County,CA


*Drop the original County & State feature column*

In [8]:
data_long = data_long.drop('County & State', axis=1)

In [9]:
data_long.head()

,Date,MedianHousePrice,County,State
0,2000-01-31,218109.6614,Los Angeles County,CA
1,2000-01-31,147964.6629,Cook County,IL
2,2000-01-31,111394.1238,Harris County,TX
3,2000-01-31,147041.8392,Maricopa County,AZ
4,2000-01-31,230036.4263,San Diego County,CA


__Check for missing values in the MedianHousePrice colum__

In [10]:
missing_values = data_long['MedianHousePrice'].isnull().sum()
total_values = len(data_long['MedianHousePrice'])
missing_percentage = (missing_values / total_values) * 100
missing_values, missing_percentage

(274895, 30.81644320882472)

*Function to apply a 5.3% deflation rate to back fill in missing MedianHousePrice value in time series data*

In [11]:
def apply_deflation(data, deflation_rate=0.053):
    """
    Apply deflation to fill in missing values in the time series data.
    We work backwards using a deflation factor.

    :param data: DataFrame with Date, MedianHousePrice columns
    :param deflation_rate: Deflation rate to be applied
    :return: DataFrame with imputed values
    """
    # Sorting the data in reverse chronological order for backward filling
    data = data.sort_values(by='Date', ascending=False)

    # Iterate through the DataFrame
    for i in range(1, len(data)):
        # If current value is missing and the previous value is present
        if pd.isna(data.iloc[i]['MedianHousePrice']) and not pd.isna(data.iloc[i - 1]['MedianHousePrice']):
            # Calculate the number of months between the dates
            months_diff = (data.iloc[i - 1]['Date'].year - data.iloc[i]['Date'].year) * 12 + \
                          data.iloc[i - 1]['Date'].month - data.iloc[i]['Date'].month

            # Apply deflation formula
            estimated_price = data.iloc[i - 1]['MedianHousePrice'] / ((1 + deflation_rate) ** months_diff)
            data.at[i, 'MedianHousePrice'] = estimated_price

    # Sorting the data back to its original chronological order
    data = data.sort_values(by='Date', ascending=True)
    return data

# Apply the deflation function to the data
data_deflated = apply_deflation(data_long)

# Display the first few rows to verify the changes
data_deflated.head()


,Date,MedianHousePrice,County,State
0,2000-01-31,218109.6614,Los Angeles County,CA
4,2000-01-31,230036.4263,San Diego County,CA
5,2000-01-31,277067.9505,Orange County,CA
6,2000-01-31,212436.6870,Kings County,NY
7,2000-01-31,127448.7814,Miami-Dade County,FL


__Check for missing values in the MedianHousePrice colum__

In [12]:
missing_values = data_deflated['MedianHousePrice'].isnull().sum()
total_values = len(data_deflated['MedianHousePrice'])
missing_percentage = (missing_values / total_values) * 100
missing_values, missing_percentage

(255918, 28.689072238913056)

*Function to apply a 5.3% deflation rate to forward fill in missing MedianHousePrice value in time series data*

In [13]:
def apply_inflation(data, inflation_rate=0.053):
    """
    Apply inflation to fill in missing values in the time series data.
    We work forward using an inflation factor.

    :param data: DataFrame with Date, MedianHousePrice columns
    :param inflation_rate: Inflation rate to be applied
    :return: DataFrame with imputed values
    """
    # Sorting the data in chronological order for forward filling
    data = data.sort_values(by='Date', ascending=True)

    # Iterate through the DataFrame
    for i in range(len(data) - 1):
        # If current value is missing and the previous value is present
        if pd.isna(data.iloc[i]['MedianHousePrice']) and not pd.isna(data.iloc[i + 1]['MedianHousePrice']):
            # Calculate the number of months between the dates
            months_diff = (data.iloc[i + 1]['Date'].year - data.iloc[i]['Date'].year) * 12 + \
                          data.iloc[i + 1]['Date'].month - data.iloc[i]['Date'].month

            # Apply inflation formula
            estimated_price = data.iloc[i + 1]['MedianHousePrice'] * ((1 + inflation_rate) ** months_diff)
            data.at[i, 'MedianHousePrice'] = estimated_price

    return data

# Apply the inflation function to the data
data_inflated = apply_inflation(data_deflated)

__Check for missing values in the MedianHousePrice colum__

In [14]:
# Check for missing values again after applying inflation
missing_values_after_inflation = data_inflated['MedianHousePrice'].isnull().sum()
total_values = len(data_inflated['MedianHousePrice'])
missing_percentage_after_inflation = (missing_values_after_inflation / total_values) * 100

missing_values_after_inflation, missing_percentage_after_inflation

(214618, 24.059235011882876)

__Exclude missing values from dataset__

*Group data by Date, County, and State and calculate missing data percentage*

In [15]:
grouped_data = data_inflated.groupby(['Date', 'County', 'State']).agg(Total=('MedianHousePrice', 'count'), Missing=('MedianHousePrice', 'sum'))
grouped_data['Missing_Percentage'] = (grouped_data['Missing'] / grouped_data['Total']) * 100

*Identify highly imcomplete segements*

In [16]:
threshold = 0.01 # threshold percentage
incomplete_segments = grouped_data[grouped_data['Missing_Percentage'] > threshold]

*Identify dates and regions to exclued*

In [17]:
dates_to_exclude = incomplete_segments.index.get_level_values('Date').unique()
counties_to_exclude = incomplete_segments.index.get_level_values('County').unique()
states_to_exclude = incomplete_segments.index.get_level_values('State').unique()

*Filter out the highly incomplete data*

In [18]:
filtered_data = data_inflated[~((data_inflated['Date'].isin(dates_to_exclude)) &
                               (data_inflated['County'].isin(counties_to_exclude)) &
                               (data_inflated['State'].isin(states_to_exclude)))]

*Visulize data and check new shape*

In [19]:
filtered_data.shape

(0, 4)

__Missing data cannot be excluded from dataset__
* Instead linear interpolation will be used to fill in missing data points

In [20]:
data_inflated_sorted = data_inflated.sort_values(by='Date') # Sort dataset by date

*Set Date feature column as the index of the DataFrame*

In [21]:
data_inflated_sorted.set_index('Date', inplace=True)

*Isolate the MedianHousePrice feature column for interpolation*

In [22]:
median_house_price = data_inflated_sorted['MedianHousePrice']

*Apply linear Interpolation*

In [23]:
interpolated_values = median_house_price.interpolate(method='linear')

*Merge the interpolated values back into the orignial DataFrame*

In [24]:
data_inflated_sorted['MedianHousePrice'] = interpolated_values

*Reset the index*

In [25]:
data_interpolated = data_inflated_sorted.reset_index()

*Check for remaining missing values*

In [26]:
remaining_missing_values = data_interpolated['MedianHousePrice'].isnull().sum()
remaining_missing_percentage = (remaining_missing_values / len(data_interpolated)) * 100

remaining_missing_values, remaining_missing_percentage

(0, 0.0)

## Feature Engineering to enhance data for effective modeling

__Extracting Year and Month__
* This will help the models to understand and leverage seasonal trends and year-over-year changes in house prices

In [27]:
data_interpolated['Year'] = data_interpolated['Date'].dt.year # Extract Year from the Date column
data_interpolated['Month'] = data_interpolated['Date'].dt.month # Extract Month from the Date column

__Creating Lagged Features__
* Past values of variables, such as previous month's house price
* These are informative in time series forecasting, as they allow models to learn from the historical trends

In [28]:
for lag in [1, 3, 6]:
    data_interpolated[f'Lag_{lag}_Month'] = data_interpolated['MedianHousePrice'].shift(lag)

__Drop missing values created by Lagging process__
* Dropping rows with missing lagged values, since they are a small portion of dataset

In [29]:
data_interpolated.dropna(subset=[f'Lag_{lag}_Month' for lag in [1, 3, 6]], inplace=True)

In [30]:
remaining_missing_values = data_interpolated['MedianHousePrice'].isnull().sum()
remaining_missing_percentage = (remaining_missing_values / len(data_interpolated)) * 100

remaining_missing_values, remaining_missing_percentage

(0, 0.0)

In [31]:
data_interpolated.head()

,Date,MedianHousePrice,County,State,Year,Month,Lag_1_Month,Lag_3_Month,Lag_6_Month
6,2000-01-31,96184.65994,Dallas County,TX,2000,1,127448.78140,277067.95050,218109.66140
7,2000-01-31,57953.66333,Riverside County,CA,2000,1,96184.65994,239875.74880,49562.45471
8,2000-01-31,260395.51300,Queens County,NY,2000,1,57953.66333,127448.78140,230036.42630
9,2000-01-31,147179.80690,Sacramento County,CA,2000,1,260395.51300,96184.65994,277067.95050
10,2000-01-31,75407.69696,Davidson County,TN,2000,1,147179.80690,57953.66333,239875.74880


## Exploratory Data Analysis

* Visualize trends over time in median house prices
* Look for seasonal patterns and variations in months

In [32]:
def plot_data_bqplot(df, title):
    # Extract numerical columns for histogram
    num_columns = df.select_dtypes(include=[np.number]).columns
    figures = []
    
    # Create histograms for each numerical column
    for col in num_columns:
        hist, edges = np.histogram(df[col].dropna(), bins=25)
        x_sc = LinearScale()
        y_sc = LinearScale()
        bar = Bars(x=edges[:-1], y=hist, scales={'x': x_sc, 'y': y_sc})
        ax_x = Axis(scale=x_sc, label=col)
        ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')
        fig = Figure(marks=[bar], axes=[ax_x, ax_y], title=f'Histogram of {col}')
        figures.append(fig)

    # Arrange the histograms in a VBox
    return VBox(figures, layout=Layout(flex_flow='row wrap', align_items='stretch', width='100%'))


https://fred.stlouisfed.org/series/MSPUS#0

## Split the data into train and test sets
* 20% for test data
* Random state = constant for model reproducibility

In [33]:
X = data_interpolated.drop(['MedianHousePrice'], axis=1)
y = data_interpolated['MedianHousePrice']
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Preserve Date Column in separate Series for later use in visualization

In [34]:
date_train = X_train_full['Date']
date_test = X_test_full['Date']

## Extract Date column from X_train and X_test for Training Models

In [35]:
X_train = X_train_full.drop('Date', axis=1)
X_test = X_test_full.drop('Date', axis=1)

## Apply Logarithm Transformations 
* To the target variable (y)
* To the numerical features with significant skew
* * Year, Month, Lag_1_Month, Lag_3_Month, Lag_6_Month

In [36]:
y_train_logged = np.log1p(y_train)
y_test_logged = np.log1p(y_test)

## Apply Logarithmic Transformations to the lag features in the test set

In [37]:
lag_features = ['Lag_1_Month', 'Lag_3_Month', 'Lag_6_Month']
X_train[lag_features] = X_train[lag_features].apply(lambda x: np.log1p(x))
X_test[lag_features] = X_test[lag_features].apply(lambda x: np.log1p(x))

## Apply the Logarithmic Transformations to the full data to be used for visualizations

In [38]:
X_train_full[lag_features] = X_train_full[lag_features].apply(lambda x: np.log1p(x))
X_test_full[lag_features] = X_test_full[lag_features].apply(lambda x: np.log1p(x))

## Visualize Data Before and After Logarithmic Transformations

In [39]:
plot_data_bqplot(X_train_full.assign(MedianHousePrice=y_train), 'Initial Data Visualization')

In [40]:
plot_data_bqplot(X_train_full.assign(LoggedMedianHousePrice=y_train_logged), 'Post-Transformation Data Visualization')

## Data Preparation for Modeling

* One-hot encode categorical variables and scale numerical features

*Select categorical and numerical features*

In [41]:
numerical_features = ['Year', 'Month', 'Lag_1_Month', 'Lag_3_Month', 'Lag_6_Month']  # already log-transformed
categorical_features = ['County', 'State']

*Create the ColumnTransformer Preprocessor with transformations for numerical and categorical data*

In [42]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

__Combine the preprocessor with a model in a pipeline__

In [43]:
linear_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

ridge_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RidgeCV(alphas=np.logspace(-6, 6, 13)))
])

lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LassoCV(alphas=np.logspace(-6, 6, 13), max_iter=10000, tol=0.01))
])

random_forest_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Train and Evaluate Linear Regression, Random Forest Regressor, Ridge Regression, and Lasso Regression models
* Train models on train dataset
* Evaluate models on test dataset
* Compare model's performance with and without ordinal formatted date included in dataset

In [44]:
# Function to fit and evaluate a given model
def fit_and_evaluate(pipeline, X_train, y_train, X_test, y_test):
    pipeline.fit(X_train, y_train)
    y_pred_logged = pipeline.predict(X_test)
    y_pred = np.expm1(y_pred_logged)  # inverse of log1p
    rmse = np.sqrt(mean_squared_error(np.expm1(y_test), y_pred))
    r2 = r2_score(np.expm1(y_test), y_pred)
    return rmse, r2

# Fit and evaluate each pipeline
linear_rmse, linear_r2 = fit_and_evaluate(linear_pipeline, X_train_full, y_train_logged, X_test_full, y_test_logged)
ridge_rmse, ridge_r2 = fit_and_evaluate(ridge_pipeline, X_train_full, y_train_logged, X_test_full, y_test_logged)
lasso_rmse, lasso_r2 = fit_and_evaluate(lasso_pipeline, X_train_full, y_train_logged, X_test_full, y_test_logged)
rf_rmse, rf_r2 = fit_and_evaluate(random_forest_pipeline, X_train_full, y_train_logged, X_test_full, y_test_logged)

print(f'Linear Regression RMSE: {linear_rmse}, R^2: {linear_r2}')
print(f'Ridge Regression RMSE: {ridge_rmse}, R^2: {ridge_r2}')
print(f'Lasso Regression RMSE: {lasso_rmse}, R^2: {lasso_r2}')
print(f'Random Forest RMSE: {rf_rmse}, R^2: {rf_r2}')

Linear Regression RMSE: 81285.68869731817, R^2: 0.5041390214455528
Ridge Regression RMSE: 81162.49960902674, R^2: 0.5056408448466331
Lasso Regression RMSE: 81339.41807421809, R^2: 0.503483282236388
Random Forest RMSE: 62125.43809758818, R^2: 0.7103519010532411


### Results of First Train and Evaluate without any model tuning
* Linear Regression RMSE: 81285.60093863156, R^2: 0.5041400921404177
* Ridge Regression RMSE: 81093.54334743644, R^2: 0.5064805104110306
* Lasso Regression RMSE: 81339.41807421809, R^2: 0.503483282236388
* Random Forest RMSE: 62125.43809758818, R^2: 0.7103519010532411

## Perform Cross-Validation Grid-Search to Optimize Random Forest Parameters & Add eXtreme Gradient Boosting model to test for further fitting

*Setup the Pipelines for Random Forest and XGBoost*

In [45]:
forest_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('forest', RandomForestRegressor(random_state=42))
])
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', random_state=42))
])

*Parameters for GridSearchCV*

In [46]:
forest_params = {
    'forest__n_estimators': [100, 200, 300],
    'forest__max_features': [None, 'sqrt', 'log2'],
    'forest__max_depth': [None, 10, 20, 30],
    'forest__min_samples_split': [2, 5, 10],
    'forest__min_samples_leaf': [1, 2, 4],
}

xgb_params = {
    'xgb__n_estimators': [100, 200, 300],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
    'xgb__max_depth': [3, 5, 7],
    'xgb__colsample_bytree': [0.5, 0.7, 1.0]
}

*Create GridSearchCV for each model*

In [47]:
forest_grid = GridSearchCV(forest_pipeline, forest_params, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
xgb_grid = GridSearchCV(xgb_pipeline, xgb_params, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

*Fit the grid search to the data*

In [ ]:
forest_grid.fit(X_train, y_train)
xgb_grid.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


__Best Models__

In [ ]:
forest_best = forest_grid.best_estimator_
xgb_best = xgb_grid.best_estimator_

## Predict and evaluate the models

*Function to evaluate models*

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = np.expm1(y_pred)  # Inverse of log1p
    rmse = np.sqrt(root_mean_squarred_error(np.expm1(y_test), y_pred))
    r2 = re_score(np.expm1(y_test), y_pred)
    return rmse, r2

*Evaluate models*

In [ ]:
forest_rmse, forest_r2 = evaluate_model(forest_best, X_test, y_test)
xbg_rmse, xgb_r2 = evaluate_model(xgb_best, X_test, y_test)

### Display the Results of the GridSeachCV for Random Forest and eXtreme Gradient Boosting models

print('Random Forest Best Params: {forest_grid.best_params_}, RMSE: {forest_rmse}, R2: {forest_r2}')
print('XGB Best Params: {xbg_grid.best_params}, RMSE: {xgb_rmse}, R2: {xgb_r2}')